# Introduction 
This project consists in analysing the last 30 years measured wind in Dublin and the possible relationship to the climate change using the synoptic station in Dublin_Airport, Co Dublin. 
Source: https://data.gov.ie/dataset/dublin-airport-hourly-data

### 1.1 Preparing the environment
First, we must prepare the necessary working environment to start the analysis problem. This introductory section is divided as follows:
- Library installation.
- Load the dataset
- Check the DF info 

First we need to explore and to anderstand the dataset, to run the program we need to import necessary libraries, like pandas and datetime.

In [ ]:

# import DataFrames
import pandas as pd

# import dates and times
import datetime as dt

# import numpy
import numpy as np

# import os 
import os 

Adding he dataset and verify if this was loaded properly.

In [ ]:
# Load the dataset
df = pd.read_csv("wind.csv")

In [ ]:
# Display the first rows
df.head()

To have more information about the dataset and check what info it's usfull for the annalysis we can use: 
- The df.shape to have: (number of rows, number of columns)
- The df.info() to obtain a summary of the DataFrame.

In [ ]:
# number of rows and columns
df.shape


In [ ]:
# info of the data frame
df.info

End